Feature engineering of the monuments dataset.


In [6]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
import geoplot
import pysal
import re
from sklearn.feature_extraction.text import CountVectorizer
from ArchaeTron.FileManager import data_saver

# from src.ArchaeTron.GeoDataHandler import TextEntitySearch

ModuleNotFoundError: No module named 'ArchaeTron'

In [2]:
sites_nw_clean = gpd.read_file('/Users/mensab/Documents/GISMapFiles/Files1600/sites_nw_clean.shp')
sites_nw_clean.columns


Index(['OID_', 'OBJECTID', 'ENTITY_ID', 'CLASS_CODE', 'SMRS', 'COUNTY',
       'TOWNLAND', 'COUNTY_ID', 'ZONE_ID', 'ITM_E', 'ITM_N', 'LATITUDE',
       'LONGITUDE', 'MONUMENT_T', 'FIRST_EDIT', 'LATEST_EDI', 'WEBSITE_LI',
       'WEB_NOTES', 'geometry'],
      dtype='object')

In [3]:
sites_nw_clean.info

<bound method DataFrame.info of         OID_  OBJECTID ENTITY_ID CLASS_CODE           SMRS   COUNTY  \
0       2325      2325   LE00212       BOCA  LE004-021----  LEITRIM   
1       5325      5325   LE00213       BOCA  LE004-022----  LEITRIM   
2       5326      5326   LE00214       BOCA  LE004-023----  LEITRIM   
3       7588      7588   LE00215       BOCA  LE004-024----  LEITRIM   
4      19549     19549   CV01168       BAWN  CV025-098002-    CAVAN   
...      ...       ...       ...        ...            ...      ...   
9480  162057    162057   SL07570       RATH  SL039-169----    SLIGO   
9481  162060    162060   SL07572       RATH  SL019-264001-    SLIGO   
9482  162061    162061   SL07573       SOUT  SL019-264002-    SLIGO   
9483  162073    162073   SL07578       RATH  SL034-266----    SLIGO   
9484  162383    162383   LE02706       COEN  LE035-046----  LEITRIM   

                                               TOWNLAND  COUNTY_ID  ZONE_ID  \
0                                   

In [4]:
sites_nw_clean['WEB_NOTES']


0       Marked only on the 1910 ed. of the OS 6-inch m...
1       Marked only on the 1910 ed. of the OS 6-inch m...
2       Marked only on the 1910 ed. of the OS 6-inch m...
3       Marked only on the 1910 ed. of the OS 6-inch m...
4       Davies (1948a, 115) recorded the site of a pla...
                              ...                        
9480    In pasture, on the level top of a low hill/dru...
9481    In good coastal grassland, 1km S of Ballysadar...
9482    In a rath (SL019-264001-), close to the NE arc...
9483    In rough, rush-grown upland pasture, located o...
9484    Reported by Jean-Charles Caillere; visible on ...
Name: WEB_NOTES, Length: 9485, dtype: object

In [5]:
# Initialize a CountVectorizer object
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2))

# Fit and transform the WEB_NOTES column
X = vectorizer.fit_transform(sites_nw_clean['WEB_NOTES'].values.astype('U'))

# Get the feature names (words or phrases)
feature_names = vectorizer.get_feature_names_out()

# Sum up the counts of each feature (word or phrase)
sum_words = X.sum(axis=0)

# Create a dictionary of words/phrases and their counts
words_freq = [(word, sum_words[0, idx]) for word, idx in zip(feature_names, list(range(X.shape[1])))]

# Sort the words/phrases by their counts
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

# Print the 10 most common words/phrases
for word, freq in words_freq[:25]:
    print(word, freq)

bank 4731
area 4719
int 3927
circular 3906
diam 3599
wth 3577
5m 2904
circular area 2825
enclosed 2644
earthen 2627
pasture 2615
raised 2475
fosse 2452
situated 2363
stone 2224
earthen bank 2208
slope 2132
facing 2094
se 2059
ne 2015
os 1975
sw 1969
low 1956
nw 1936
area int 1857


In [17]:
# I wish to see sample WEB_NOTES entries that have the word diam or wth in them

# Create a list of words/phrases to search for
search_words = ['diam', 'wth']

# Create a list of indices where the search words/phrases are found
search_indices = []
for i, note in enumerate(sites_nw_clean['WEB_NOTES']):
    if any(word in note for word in search_words):
        search_indices.append(i)

# Display sample entries based on the found indices
sample_entries = sites_nw_clean.loc[search_indices, 'WEB_NOTES']
print(sample_entries)

5       Raised circular area (int. diam 37.5m) enclose...
11      On a broad rocky ridge in otherwise low-lying ...
13      Raised circular area (int. diam. 54m) enclosed...
15      Raised oval area (int. diam. 34.2m) enclosed b...
17      Raised circular area (int. diam. 45m) enclosed...
                              ...                        
9472    Reported (5-10-2021) by Mr Aidan Harte, Projec...
9475    Situated in the S part of the subrectangular C...
9476    A small island (diam. c. 10m) is depicted only...
9477    Located in a small U-shaped bay (dims c. 90m E...
9484    Reported by Jean-Charles Caillere; visible on ...
Name: WEB_NOTES, Length: 3595, dtype: object


In [ ]:
# print these entries to a text file for further analysis



In [11]:
# Use regex to extract both "diam" and "wth" and their respective diameters

pattern_diam = r'diam(?:eter)?\s*([\d.]+)'
pattern_wth = r'wth(?:\s*is)?\s*([\d.]+)'

sites_nw_clean['Diameter_diam'] = sites_nw_clean['WEB_NOTES'].str.extract(pattern_diam, flags=re.IGNORECASE)
sites_nw_clean['Diameter_wth'] = sites_nw_clean['WEB_NOTES'].str.extract(pattern_wth, flags=re.IGNORECASE)




In [12]:
#  Convert the 'Diameter_diam' and 'Diameter_wth' columns to numeric (float) type

sites_nw_clean['Diameter_diam'] = pd.to_numeric(sites_nw_clean['Diameter_diam'], errors='coerce')
sites_nw_clean['Diameter_wth'] = pd.to_numeric(sites_nw_clean['Diameter_wth'], errors='coerce')



In [13]:
# Sum the diameter values from "diam" and "wth" and create a new 'Diameter' column
sites_nw_clean['Diameter'] = sites_nw_clean['Diameter_diam'] + sites_nw_clean['Diameter_wth']


# Replace NaN values in 'Diameter' with 0
sites_nw_clean['Diameter'].fillna(0, inplace=True)

In [14]:
# Drop the intermediate columns
sites_nw_clean.drop(columns=['Diameter_diam', 'Diameter_wth'], inplace=True)


In [15]:
# Display the result
sites_nw_clean['Diameter']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9480    0.0
9481    0.0
9482    0.0
9483    0.0
9484    0.0
Name: Diameter, Length: 9485, dtype: float64

In [ ]:
# Create a function to find words in a text, or import those ive alreafy written


def find_words(text):
    words_to_find = ['univallate', 'bivallate','trivallate','bi-vallate', 'tri-vallate', 'single ditch', 'double ditch', 'triple ditch', 'single bank', 'double bank', 'triple bank', 'single rampart', 'double rampart', 'triple rampart', 'triple enclosure']
    found_words = [word for word in words_to_find if word in text]
    return ', '.join(found_words)


# Apply the function to create a new column 'found_words'
sites_nw_clean['found_words'] = sites_nw_clean['WEB_NOTES'].apply(find_words)

In [ ]:
sites_nw_clean['found_words'].value_counts()

In [ ]:
# find row with 'trivallate' in 'found_words' column
trivallate = sites_nw_clean[sites_nw_clean['found_words'] == 'trivallate']
trivallate.head()

In [ ]:
pd.DataFrame(filtered_points_2['MONUMENT_T'].value_counts()).plot.bar(width=0.5, figsize=(15,6))
plt.show()